# Dane techniczne sprzętu

Obliczenia zostały wykonane na komputerze o następujących parametrach:

- Procesor: AMD Ryzen $7$ $4700$U ($8$ rdzeni, $8$ wątków),

- Pamięć RAM: $16$ GB $3200$ MHz

# Biblioteki

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import random
import time
from math import sqrt

In [2]:
sns.set(rc={"figure.dpi": 200, 'savefig.dpi': 200})

# Pomocnicze funkcje

## Obliczenia

### Budowanie macierzy wypełnionej zgodnie ze wzorem

#### Macierz kwadratowa

In [3]:
def create_matrix(fn, n, m, dtype=np.float64):
    A = np.empty((n, m), dtype)
    
    for i in range(n):
        for j in range(m):
            A[i, j] = fn(i + 1, j + 1)
            
    return A

#### Macierz o 3 przekątnych (jako 3 kolumny)
##### (Taki sposób przechowywania macierzy pozwala na zmniejszenie złożoności pamięciowej algorytmu Thomasa)

In [4]:
def create_thomas_matrix(fn, n, _=None, dtype=np.float64):
    A = np.empty((n, 3), dtype)
    
    for i in range(n):
        for k in range(3):
            j = i + k - 1
            if j < 0 or j >= n:
                A[i, k] = 0
            else:
                A[i, k] = fn(i + 1, j + 1)
            
    return A

### Tworzenie wektorów

#### Tworzenie wektora wypełnionego wartościami zgodnie z zadaną funkcją

In [5]:
def create_vector(fn, n, dtype=np.float64):
    return np.array([fn(i) for i in range(n)], dtype)

### Pomiar czasu wykonywania kodu

#### Dekorator, pozwalający na pomiar czasu wykonania kodu

In [6]:
def print_duration(sec):
    t = int(sec)
    ms = int((sec - t) * 1000)
    s = t % 60
    t //= 60
    m = t % 60
    t //= 60
    print(f'Łączny czas wykonania: {t:>02}:{m:>02}:{s:>02}.{ms:<03}')

def timed(*, save_fn=None, save_dict=None):
    def decorator(fn):
        def inner(*args, **kwargs):
            start_time = time.perf_counter()
            res = fn(*args, **kwargs)
            duration = time.perf_counter() - start_time
    
            if save_dict is not None and save_fn: 
                save_dict[save_fn(*args, **kwargs)] = duration
            return res
        
        return inner
    return decorator

# Wstęp

Dany jest układ równań liniowych $Ax=b$. 

Elementy macierzy $A$ są zadane wzorem ($m,k$ - parametry zadania podane indywidualnie):

$
\begin{cases} 
    a_{i, i} = k \\ 
    a_{i, j} = (-1)^j \frac{m}{j} & dla & j > 1 \\
    a_{i, i-1} = \frac{m}{i} \\
    a_{i, j} = 0 & dla & j < i - 1 \\
\end{cases}
$

gdzie $k=11, m=2$

Przyjmij wektor $x$ jako dowolną n-elementową permutację ze zbioru $\{1, -1\}$ i oblicz wektor $b$. 

## Implementacja funkcji wyznaczającej elementy macierzy

In [7]:
k = 11
m = 2

def fn(i, j):
    if i == j: return k
    if j < i: return (-1)**j * m / j
    if j == i - 1: return m / i
    return 0

## Tworzenie macierzy $A$

Poniższa funkcja ma jedynie na celu ułatwienie tworzenia macierzy $A$ (żeby za każdym razem nie trzeba było przekazywać wszystkich parametrów).

In [8]:
create_A_matrix = lambda n, dtype=np.float64: create_matrix(fn, n, n, dtype)

###### Testy

In [9]:
create_A_matrix(5)

array([[11.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-2.        , 11.        ,  0.        ,  0.        ,  0.        ],
       [-2.        ,  1.        , 11.        ,  0.        ,  0.        ],
       [-2.        ,  1.        , -0.66666667, 11.        ,  0.        ],
       [-2.        ,  1.        , -0.66666667,  0.5       , 11.        ]])

## Wyznaczanie wektora $x$

Ponieważ, jako wektor $x$, będący rozwiązaniem układu równań $Ax=b$, możemy przyjąć wektor o współrzędnych, będących dowolną $n$-elementową permutacją liczb ze zbioru $\{-1, 1\}$, przyjmuję, że wektor $x$ będzie postaci $[-1, 1, -1, 1, ...]$ (na przemian będą występowały liczby $-1$ oraz $1$, rozpoczynając się od $-1$)

In [10]:
create_x_vector = lambda n, dtype=np.float64: create_vector(lambda i: (-1)**(i + 1), n, dtype)

###### Test

In [11]:
create_x_vector(5)

array([-1.,  1., -1.,  1., -1.])

## Obliczanie wektora $b$

Ponieważ nie znamy wektora $b$, w celu jego obliczenia dla danego $n$, konieczne jest najpierw wyznaczenie wektora $x$, który ma być rozwiązaniem układu równań. Następnie, korzystając z tego wektora, należy obliczyć wektor $b$. W późniejszych obliczeniach będziemy ten wektor wykorzystywać wraz z macierzą $A$ do obliczenia wektora $x$, po czym, porównamy wartość obliczonego w ten sposób wektora $x$ z wartością wektora $x$ wcześniej przyjętego jako rozwiązanie układu.

In [12]:
n = 5
A = create_A_matrix(n)
x_solution = create_x_vector(n, A.dtype)
b = A @ x_solution
b

array([-11.        ,  13.        ,  -8.        ,  14.66666667,
        -6.83333333])

# Zadanie 1

## Opis zadania

Metodą Jacobiego rozwiąż układ równań liniowych $Ax=b$ (przyjmując jako niewiadomą wektor $x$), 
przyjmując kolejno kryterium stopu: 

- $||x^{(i+1)} - x^{(i)}|| < \rho$

- $||Ax^{(i)} - b|| < \rho$

Obliczenia wykonaj dla różnych rozmiarów układu $n$, dla różnych wektorów początkowych, a także 
różnych wartości $\rho$ w kryteriach stopu. (*Podaj, jak liczono normę.*) Wyznacz liczbę iteracji oraz 
sprawdź różnicę w czasie obliczeń dla obu kryteriów stopu. 
Sprawdź dokładność obliczeń. 

## Kryteria stopu

### Sposób obliczania normy

Przy obliczaniu kryteriów stopu, wykorzystałem normę Euklidesową. Sposób wyznaczania tej normy w $n$-wymiarowej przestrzeni wektorowej przedstawia poniższy wzór:

$||x|| = \sqrt{\sum_{i}^{n} x_i^2}$

In [13]:
euclidean_norm = lambda x: sqrt(sum(xi**2 for xi in x))

### Kryteria

#### Dekorator, ułatwiający korzystanie z kryteriów

In [14]:
def apply_function_args(fn):
    def inner(**kwargs):
        arg_names = fn.__code__.co_varnames
        return fn(**{k: kwargs[k] for k in arg_names})
    return inner

#### 1. Kryterium

$||x^{(i+1)} - x^{(i)}|| < \rho$

In [15]:
@apply_function_args
def progressive_difference(x_curr, x_next, 𝜌):
    return euclidean_norm(x_next - x_curr) < 𝜌

###### Test

In [16]:
progressive_difference(
    foo=12412312,  # @apply_function_args decorator ensures that redundant parameters will be filtered out
    x_curr=np.array([3, 3, 3]),
    x_next=np.array([1, 1, 1]),
    𝜌=5.1
)

True

#### 2. Kryterium

$||Ax^{(i)} - b|| < \rho$

In [17]:
@apply_function_args
def solution_difference(x_curr, A, b, 𝜌):
    return euclidean_norm(A @ x_curr - b) < 𝜌

In [18]:
solution_difference(
    x_curr=np.array([3, 3, 3]),
    A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    b = np.array([1, 1, 1]),
    𝜌=123
)

True

## Wyznaczanie błędów obliczeń

### Norma maksimum

Do wyznaczania błędów skorzystałem z normy maksimum, przy pomocy której wyznaczałem maksymalną bezwzględną różnicę wartości współrzędnych wektorów (porównując zadany wektor $x$ z obliczonym).

In [19]:
max_err = lambda v1, v2: max(abs(v1 - v2))

## Metoda Jacobiego

Umieszczony poniżej słownik służy do zapisywania czasów wykonywanych obliczeń.

In [20]:
times = {}

In [34]:
@timed(
    save_fn=lambda A, *args, **kwargs: len(A),
    save_dict=times
)
def jacobi(A, b, start_vector, *, stop_criterion=progressive_difference, 𝜌=1e-6, dtype=np.float64, max_iters=1000):
    n = len(A)
    x_curr = start_vector
    x_next = np.empty(n, dtype)
    iters = 0
    
    print()
    while iters < max_iters:
        iters += 1
        for i in range(n):
            x_next[i] = 1 / A[i, i] * (b[i] - sum(A[i, j] * x_curr[j] for j in range(n) if j != i))
            
        print(iters, max_err(x_curr, create_x_vector(n)))
        if stop_criterion(A=A, b=b, x_curr=x_curr, x_next=x_next, 𝜌=𝜌): break
        x_curr = x_next[:]
    else:
        iters = float('inf')
        
    return x_next, iters

###### Testy

Jak możemy zauważyć, wybór wektora początkowego ma znikome znaczenie

In [35]:
n = 5
A = create_matrix(fn, n, n)
x_solution = create_x_vector(n)
b = A @ x_solution

start_x = np.array([random.random() * 10e20 for _ in range(n)])
x, iters = jacobi(A, b, start_x)
start_x, x, iters


1 8.858426178461574e+20
2 0.0


(array([4.98713446e+20, 4.07235065e+19, 6.29058978e+20, 8.85842618e+20,
        4.57114270e+20]),
 array([-1.,  1., -1.,  1., -1.]),
 2)

## Rozwiązanie

In [36]:
times.clear()

### Pomocnicza funkcja

In [37]:
def calculate(n_list, start_vector_fns, 𝜌_list, stop_criterion, dtype=np.float64, max_iters=1000):
    results = {}
    
    for n in n_list:
        A = create_A_matrix(n, dtype)
        x_solution = create_x_vector(n, dtype)
        b = A @ x_solution
        
        calculated = [[None] * len(𝜌_list) for _ in range(len(start_vector_fns))]
        start_vectors = [[None] * len(𝜌_list) for _ in range(len(start_vector_fns))]
        errors = np.empty((len(start_vector_fns), len(𝜌_list)))
        iters = np.empty((len(start_vector_fns), len(𝜌_list)))
        times_ = np.empty((len(start_vector_fns), len(𝜌_list)))
            
        for i, (fn_name, start_vector_fn) in enumerate(start_vector_fns.items()):
            x_start = create_vector(start_vector_fn, n, dtype)
            
            for j, 𝜌 in enumerate(𝜌_list):
                x, it = jacobi(A, b, x_start, stop_criterion=stop_criterion, 𝜌=𝜌, dtype=dtype, max_iters=max_iters)
                calculated[i][j] = x
                start_vectors[i][j] = x_start
                errors[i, j] = max_err(x, x_solution)
                #print(list(x), list(x_solution), errors[i, j])
                iters[i, j] = it
                times_[i, j] = times[n]
        
        # Save results
        results[n] = {}
        for key, data in zip(
            ('calculated', 'errors', 'iters', 'times', 'start_vectors'), 
            (calculated, errors, iters, times_, start_vectors)
        ):
            results[n][key] = pd.DataFrame(data, index=start_vector_fns.keys(), columns=𝜌_list)
            
    return results

###### Testy

In [38]:
n_list = [10, 25, 100]
start_vector_fns = {
    'sq': lambda i: i ** 2,
    'random': lambda i: random.random(),
    'randint': lambda i: random.randint(-10e10, 10e10)
}
𝜌_list = [
    1e-5,
    1e-10,
    1e-15
]
stop_criterion = progressive_difference

results = calculate(n_list, start_vector_fns, 𝜌_list, stop_criterion)


1 80.0
2 3.3306690738754696e-16

1 80.0
2 3.3306690738754696e-16

1 80.0
2 3.3306690738754696e-16

1 1.8575244340417663
2 3.3306690738754696e-16

1 1.8575244340417663
2 3.3306690738754696e-16

1 1.8575244340417663
2 3.3306690738754696e-16

1 96227135821.0
2 3.3306690738754696e-16

1 96227135821.0
2 3.3306690738754696e-16

1 96227135821.0
2 3.3306690738754696e-16

1 577.0
2 3.3306690738754696e-16

1 577.0
2 3.3306690738754696e-16

1 577.0
2 3.3306690738754696e-16

1 1.8057810115716388
2 3.3306690738754696e-16

1 1.8057810115716388
2 3.3306690738754696e-16

1 1.8057810115716388
2 3.3306690738754696e-16

1 94819114311.0
2 3.3306690738754696e-16

1 94819114311.0
2 3.3306690738754696e-16

1 94819114311.0
2 3.3306690738754696e-16

1 9800.0
2 4.440892098500626e-16

1 9800.0
2 4.440892098500626e-16

1 9800.0
2 4.440892098500626e-16

1 1.999303850440168
2 4.440892098500626e-16

1 1.999303850440168
2 4.440892098500626e-16

1 1.999303850440168
2 4.440892098500626e-16

1 99575017225.0
2 4.4408920

Ponownie widzimy, że liczba iteracji jest niezależna od wektora początkowego

In [39]:
results[100]['start_vectors']

,1.000000e-05,1.000000e-10,1.000000e-15
sq,"[0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 6...","[0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 6...","[0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 6..."
random,"[0.28426519455673327, 0.7721000189647086, 0.23...","[0.28426519455673327, 0.7721000189647086, 0.23...","[0.28426519455673327, 0.7721000189647086, 0.23..."
randint,"[73707888629.0, -59086766517.0, 49335938451.0,...","[73707888629.0, -59086766517.0, 49335938451.0,...","[73707888629.0, -59086766517.0, 49335938451.0,..."


In [40]:
results[100]['iters']

,1.000000e-05,1.000000e-10,1.000000e-15
sq,2.0,2.0,2.0
random,2.0,2.0,2.0
randint,2.0,2.0,2.0


### Dla ustalonego $n$ oraz różnych $\rho$ i różnych wektorów początkowych

Jak widzimy, nawet dla bardzo dużej precyzji obliczeń i dużej wartości $n$, musimy wykonać tylko $2$ iteracje, w celu obliczenia wektora $x$.

#### Dla 1. kryterium stopu

In [41]:
n_list = [1_000]
start_vector_fns = {
    'sq': lambda i: i ** 2,
    'random': lambda i: random.random(),
    'randint': lambda i: random.randint(-10e10, 10e10)
}
𝜌_list = [
    1e-5,
    1e-10,
    1e-15
]
stop_criterion = progressive_difference

results = calculate(n_list, start_vector_fns, 𝜌_list, stop_criterion)


1 998000.0
2 1.4432899320127035e-15

1 998000.0
2 1.4432899320127035e-15

1 998000.0
2 1.4432899320127035e-15

1 1.9985779151791996
2 1.4432899320127035e-15

1 1.9985779151791996
2 1.4432899320127035e-15

1 1.9985779151791996
2 1.4432899320127035e-15

1 99989593223.0
2 1.4432899320127035e-15

1 99989593223.0
2 1.4432899320127035e-15

1 99989593223.0
2 1.4432899320127035e-15


In [42]:
results[1_000]['iters']

,1.000000e-05,1.000000e-10,1.000000e-15
sq,2.0,2.0,2.0
random,2.0,2.0,2.0
randint,2.0,2.0,2.0


In [45]:
results[1_000]['start_vectors']

,1.000000e-05,1.000000e-10,1.000000e-15
sq,"[0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 6...","[0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 6...","[0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 6..."
random,"[0.6841054086348236, 0.6383651657816994, 0.548...","[0.6841054086348236, 0.6383651657816994, 0.548...","[0.6841054086348236, 0.6383651657816994, 0.548..."
randint,"[25917682819.0, 65233590039.0, -45852413076.0,...","[25917682819.0, 65233590039.0, -45852413076.0,...","[25917682819.0, 65233590039.0, -45852413076.0,..."


In [46]:
results[1_000]['errors']

,1.000000e-05,1.000000e-10,1.000000e-15
sq,1.443290e-15,1.443290e-15,1.443290e-15
random,1.443290e-15,1.443290e-15,1.443290e-15
randint,1.443290e-15,1.443290e-15,1.443290e-15


#### Dla 2. kryterium stopu

In [47]:
n_list = [1_000]
start_vector_fns = {
    'sq': lambda i: i ** 2,
    'random': lambda i: random.random(),
    'randint': lambda i: random.randint(-10e10, 10e10)
}
𝜌_list = [
    1e-5,
    1e-10,
    1e-15
]
stop_criterion = solution_difference

results = calculate(n_list, start_vector_fns, 𝜌_list, stop_criterion)


1 998000.0
2 1.4432899320127035e-15

1 998000.0
2 1.4432899320127035e-15

1 998000.0
2 1.4432899320127035e-15
3 1.4432899320127035e-15
4 1.4432899320127035e-15
5 1.4432899320127035e-15
6 1.4432899320127035e-15
7 1.4432899320127035e-15
8 1.4432899320127035e-15
9 1.4432899320127035e-15
10 1.4432899320127035e-15
11 1.4432899320127035e-15
12 1.4432899320127035e-15
13 1.4432899320127035e-15
14 1.4432899320127035e-15
15 1.4432899320127035e-15
16 1.4432899320127035e-15
17 1.4432899320127035e-15
18 1.4432899320127035e-15
19 1.4432899320127035e-15
20 1.4432899320127035e-15
21 1.4432899320127035e-15
22 1.4432899320127035e-15
23 1.4432899320127035e-15
24 1.4432899320127035e-15
25 1.4432899320127035e-15
26 1.4432899320127035e-15
27 1.4432899320127035e-15
28 1.4432899320127035e-15
29 1.4432899320127035e-15
30 1.4432899320127035e-15
31 1.4432899320127035e-15
32 1.4432899320127035e-15
33 1.4432899320127035e-15
34 1.4432899320127035e-15
35 1.4432899320127035e-15
36 1.4432899320127035e-15
37 1.4432899

KeyboardInterrupt: 

In [ ]:
results[1_000]['iters']

In [ ]:
results[1_000]['start_vectors']

In [ ]:
results[1_000]['errors']